In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [187]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])for i in range(len(test_sentence) - 2)]

# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [188]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [189]:
#vocab

In [190]:
#word_to_ix

In [191]:
len(vocab)

97

In [192]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [193]:
NGramLanguageModeler

__main__.NGramLanguageModeler

In [194]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [195]:
model

NGramLanguageModeler(
  (embeddings): Embedding(97, 10)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)

In [196]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[523.2276015281677, 520.9332156181335, 518.6547837257385, 516.3902449607849, 514.13849401474, 511.89990425109863, 509.6728949546814, 507.4568190574646, 505.252055644989, 503.0566794872284]


In [197]:
[word_to_ix[target]]

[75]

## CBOW

In [230]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

# create your model and train.  here are some functions to help you make
# the data ready for use by your module

In [231]:
word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
    
ix_to_word = {ix:word for word,ix in word_to_ix.items()}

In [233]:
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [234]:
context

['the', 'computer', 'our', 'spells.']

In [235]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [236]:
make_context_vector(data[0][0], word_to_ix)

tensor([ 8, 34,  7, 15])

In [237]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds_sum = embeds.sum(dim=0)
        out = F.relu(self.linear1(embeds_sum))
        out = self.linear2(out)
        out = out.view(1,-1)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [238]:
vocab_size = len(vocab)
embedding_dim = 10
context_size = CONTEXT_SIZE * 2

In [239]:
cbow = CBOW(vocab_size, embedding_dim, context_size)

In [221]:
inputs = make_context_vector(data[0][0], word_to_ix)

In [222]:
cbow.forward(inputs).shape

torch.Size([1, 49])

In [223]:
for context,target_word in data:
    inputs = make_context_vector(context, word_to_ix)
    target = torch.tensor([word_to_ix[target_word]], dtype=torch.long)

In [224]:
losses = []
loss_function = nn.NLLLoss()
optimizer = optim.SGD(cbow.parameters(), lr=0.001)

In [225]:
log_probs = cbow(inputs)

loss = loss_function(log_probs, target)

In [226]:
log_probs

tensor([[-5.3302, -3.6561, -3.3036, -4.5270, -3.7221, -4.3553, -4.8422, -3.5539,
         -2.7043, -4.2222, -3.4277, -5.1106, -4.1313, -3.2672, -4.6400, -4.3759,
         -4.0656, -4.0411, -4.3058, -3.9493, -3.7954, -3.6926, -4.2592, -3.3908,
         -4.1153, -3.4055, -4.2365, -3.7171, -4.0125, -3.3853, -3.6832, -5.2476,
         -3.3941, -3.4589, -4.1281, -4.6407, -3.5894, -4.6401, -3.9302, -4.2185,
         -4.3847, -3.3234, -5.0479, -3.9069, -4.9506, -3.4299, -4.0739, -4.2901,
         -4.5288]], grad_fn=<LogSoftmaxBackward>)

In [227]:
target

tensor([4])

In [228]:
losses = []
for epoch in range(50):
    total_loss = 0
    for context,target_word in data:
        inputs = make_context_vector(context, word_to_ix)
        target = torch.tensor([word_to_ix[target_word]], dtype=torch.long)
        cbow.zero_grad()
        
        log_probs = cbow(inputs)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(round(total_loss,2))
print(losses)

[229.59, 225.52, 221.57, 217.73, 214.0, 210.38, 206.87, 203.45, 200.13, 196.9, 193.75, 190.7, 187.72, 184.83, 182.0, 179.26, 176.57, 173.95, 171.39, 168.88, 166.43, 164.03, 161.68, 159.38, 157.11, 154.89, 152.71, 150.56, 148.45, 146.37, 144.32, 142.3, 140.31, 138.35, 136.42, 134.51, 132.62, 130.76, 128.92, 127.11, 125.31, 123.54, 121.79, 120.06, 118.35, 116.66, 114.99, 113.34, 111.71, 110.1]


In [229]:
for context,target_word in data:
    inputs = make_context_vector(context, word_to_ix)
    target = torch.tensor([word_to_ix[target_word]], dtype=torch.long)
    cbow.zero_grad()

    log_probs = cbow(inputs)
    
    argmax = int(log_probs.argmax())
    prediction = ix_to_word[argmax]
    
    print(f'{context[0]} {context[1]} [{target_word}] {context[2]} {context[3]} ==> {prediction}')

We are [about] to study ==> about
are about [to] study the ==> to
about to [study] the idea ==> study
to study [the] idea of ==> the
study the [idea] of a ==> the
the idea [of] a computational ==> of
idea of [a] computational process. ==> a
of a [computational] process. Computational ==> computational
a computational [process.] Computational processes ==> process.
computational process. [Computational] processes are ==> Computational
process. Computational [processes] are abstract ==> processes
Computational processes [are] abstract beings ==> are
processes are [abstract] beings that ==> abstract
are abstract [beings] that inhabit ==> manipulate
abstract beings [that] inhabit computers. ==> that
beings that [inhabit] computers. As ==> of
that inhabit [computers.] As they ==> a
inhabit computers. [As] they evolve, ==> As
computers. As [they] evolve, processes ==> they
As they [evolve,] processes manipulate ==> evolve,
they evolve, [processes] manipulate other ==> processes
evolve, proce